KPO Insurance Clamis DashBoard

In [1]:
## import panda as pd

# install panda if not installed

!pip install pandas openpyxl

!pip install pyngrok

!pip install streamlit pandas plotly openpyxl pyngrok

!pip install streamlit


In [2]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import os  # For file cleanup

st.set_page_config(page_title="Insurance Claims Dashboard", layout="wide")
st.title("Insurance Claims Analysis and Visualization Dashboard")

# File uploader
uploaded_file = st.file_uploader("Upload File", type=["csv", "xls", "xlsx", "xlsm"])

if uploaded_file:
    try:
        # Load data
        if uploaded_file.name.endswith(".csv"):
            df = pd.read_csv(uploaded_file)
        else:
            df = pd.read_excel(uploaded_file, engine='openpyxl')

        # Basic validation: Check for required columns
        required_cols = ['DATE_OF_SERVICE', 'SUBMITTED_AMOUNT', 'INSURANCE / TPA', 'DOCTOR_NAME']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            st.error(f"Missing required columns: {missing_cols}. Please check your file.")
            st.stop()

        # Data cleaning
        df.columns = df.columns.str.strip()
        df['DATE_OF_SERVICE'] = pd.to_datetime(df['DATE_OF_SERVICE'], errors='coerce')
        df = df.dropna(subset=['DATE_OF_SERVICE'])

        # Amount columns (with validation)
        amount_cols = ['SUBMITTED_AMOUNT', 'ReSubmitted_Amount_1', 'ReSubmitted_Amount2',
                       'PAID_AMOUNT', 'ReSubmission_Paid_Amount_1', 'ReSubmission_Paid_Amount2',
                       'DENIED_AMOUNT', 'RESUBMISSION_DENIED_AMOUNT_RA_1', 'RESUBMISSION_DENIED_AMOUNT_RA_2']
        df[amount_cols] = df[amount_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

        # Derived columns
        df['Year'] = df['DATE_OF_SERVICE'].dt.year
        df['Month'] = df['DATE_OF_SERVICE'].dt.strftime('%b')
        df['Quarter'] = df['DATE_OF_SERVICE'].dt.quarter

        # Simplified calculations (adjust logic as needed for accuracy)
        df['Total Submitted Amount'] = round(df['SUBMITTED_AMOUNT'] + df['ReSubmitted_Amount_1'] + df['ReSubmitted_Amount2'],2)

        df['Total Paid Amount'] = round(df['PAID_AMOUNT'] + df['ReSubmission_Paid_Amount_1'] + df['ReSubmission_Paid_Amount2'],2)

        df['Total Denied Amount'] = round((df['DENIED_AMOUNT'] - df['ReSubmitted_Amount_1']) + (df['RESUBMISSION_DENIED_AMOUNT_RA_1'] - df['ReSubmitted_Amount2']) + df['RESUBMISSION_DENIED_AMOUNT_RA_2'],2)

        df['Total Pending Amount'] = round(df['SUBMITTED_AMOUNT'] - (df['Total Paid Amount'] + df['Total Denied Amount']),2)

        # Sidebar filters for interactivity
        st.sidebar.header("Filters")

        # Options for doctors with "ALL" added
        doctor_options = ["ALL"] + list(df['DOCTOR_NAME'].unique())
        selected_doctor = st.sidebar.multiselect("Select Doctor(s)", options=doctor_options, default=["ALL"])

        # Options for years with "ALL" added
        year_options = ["ALL"] + sorted(df['Year'].unique())
        selected_year = st.sidebar.multiselect("Select Year(s)", options=year_options, default=["ALL"])

        # Options for insurance with "ALL" added
        insurance_options = ["ALL"] + list(df['INSURANCE / TPA'].unique())
        selected_insurance = st.sidebar.multiselect("Select Insurance(s)", options=insurance_options, default=["ALL"])

        # Determine effective selections: if "ALL" is selected, use all options; otherwise, use selected ones
        effective_doctor = df['DOCTOR_NAME'].unique() if "ALL" in selected_doctor else [d for d in selected_doctor if d != "ALL"]
        effective_year = df['Year'].unique() if "ALL" in selected_year else [y for y in selected_year if y != "ALL"]
        effective_insurance = df['INSURANCE / TPA'].unique() if "ALL" in selected_insurance else [i for i in selected_insurance if i != "ALL"]

        # Apply filters to raw df
        filtered_df = df[
            (df['DOCTOR_NAME'].isin(effective_doctor)) &
            (df['Year'].isin(effective_year)) &
            (df['INSURANCE / TPA'].isin(effective_insurance))
        ]

        # Grouped Summary Table (filtered)
        grouped_summary = filtered_df.groupby(['Year', 'INSURANCE / TPA']).agg(
            Claimed_Amount=('Total Submitted Amount', 'sum'),
            Received_Amount=('Total Paid Amount', 'sum'),
            Denied_Amount=('Total Denied Amount', 'sum'),
            Pending_Amount=('Total Pending Amount', 'sum')
        ).reset_index().sort_values(by='Year')

        # Submitted per Month Table (filtered)
        month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        grouped_submitted = filtered_df.groupby(['Year', 'INSURANCE / TPA', 'Month'])['SUBMITTED_AMOUNT'].sum().unstack().fillna(0)
        grouped_submitted = grouped_submitted.reindex(columns=month_order, fill_value=0).reset_index()

        # Doctor + Insurance + Month Table (Pivot, filtered)
        doctor_insurance_month = filtered_df.groupby(['Year', 'DOCTOR_NAME', 'INSURANCE / TPA', 'Month']).agg(
            Claimed_Amount=('Total Submitted Amount', 'sum'),
            Received_Amount=('Total Paid Amount', 'sum'),
            Denied_Amount=('Total Denied Amount', 'sum'),
            Pending_Amount=('Total Pending Amount', 'sum')
        ).reset_index()
        doctor_insurance_month['Month'] = pd.Categorical(doctor_insurance_month['Month'], categories=month_order, ordered=True)
        doctor_insurance_month = doctor_insurance_month.sort_values(['Year', 'DOCTOR_NAME', 'INSURANCE / TPA', 'Month'])
        pivot_table = doctor_insurance_month.pivot_table(
            index=['Year', 'DOCTOR_NAME', 'INSURANCE / TPA'],
            columns='Month',
            values=['Claimed_Amount', 'Received_Amount', 'Denied_Amount', 'Pending_Amount'],
            fill_value=0
        )
        pivot_table.columns = [f'{month}_{metric.upper()}' for metric, month in pivot_table.columns]
        doctor_insurance_month = pivot_table.reset_index()

        # Display Tables
        st.subheader("Summary Table: Claimed, Received, Denied, Pending (Filtered)")
        st.dataframe(grouped_summary)
        st.subheader("Submitted Amount Per Month (Insurance/TPA wise, Filtered)")
        st.dataframe(grouped_submitted)
        st.subheader("Doctor-wise | Insurance-wise | Monthly Summary Table (Filtered)")
        st.dataframe(doctor_insurance_month)


        # Charts
        st.subheader("Data Visualizations (Filtered Data)")

        # Aggregate by Doctor for bar chart (sum across all filtered data)
        doctor_summary = filtered_df.groupby('DOCTOR_NAME').agg(
            Claimed_Amount=('Total Submitted Amount', 'sum'),
            Received_Amount=('Total Paid Amount', 'sum'),
            Denied_Amount=('Total Denied Amount', 'sum'),
            Pending_Amount=('Total Pending Amount', 'sum')
        ).reset_index()

        # Consolidated Bar Chart: By Doctor (Amounts)
        bar_chart = px.bar(doctor_summary, x='DOCTOR_NAME', y=['Claimed_Amount', 'Received_Amount', 'Denied_Amount', 'Pending_Amount'],
                           title="Claim vs Received vs Denied vs Pending by Doctor",
                           labels={'value': 'Amount', 'variable': 'Category'}, barmode='group')
        st.plotly_chart(bar_chart)

        # Consolidated Bar Chart: Total claims (count of rows) by doctor
        doctor_claims_count = filtered_df.groupby('DOCTOR_NAME').agg(Claimed_Amount=('DOCTOR_NAME', 'size')).reset_index()
        bar_chart = px.bar(doctor_claims_count, x='DOCTOR_NAME', y='Claimed_Amount',
                           color_discrete_sequence=px.colors.qualitative.Set2,
                           title="Total Claimed Amount (Count of Claims) per Doctor",
                           labels={'Claimed_Amount': 'Number of Claims', 'DOCTOR_NAME': 'Doctor Name'})
        st.plotly_chart(bar_chart)

        # Scatter: Doctor Performance (Claims Count vs. Paid Amount)
        doctor_performance = filtered_df.groupby('DOCTOR_NAME').agg(
            Total_Claims=('DOCTOR_NAME', 'size'),
            Total_Paid=('Total Paid Amount', 'sum')
        ).reset_index()
        scatter_chart = px.scatter(doctor_performance, x='Total_Claims', y='Total_Paid',
                                   color='DOCTOR_NAME', size='Total_Claims',
                                   title="Doctor Performance: Claims Count vs. Paid Amount",
                                   labels={'Total_Claims': 'Number of Claims', 'Total_Paid': 'Total Paid Amount'})
        st.plotly_chart(scatter_chart)

        # Bar: By Year
        bar_chart_year = px.bar(grouped_summary, x='Year', y=['Claimed_Amount', 'Received_Amount', 'Denied_Amount'],
                                title="Total Claimed vs Paid vs Denied Amount by Year",
                                labels={'value': 'Amount', 'variable': 'Category'}, barmode='group')
        st.plotly_chart(bar_chart_year)

        # Consolidated Bar: Claimed vs Paid by Insurance
        insurance_summary = grouped_summary.groupby('INSURANCE / TPA').agg(
            Claimed_Amount=('Claimed_Amount', 'sum'),
            Received_Amount=('Received_Amount', 'sum')
        ).reset_index()
        bar_insurance = px.bar(insurance_summary, x='INSURANCE / TPA', y=['Claimed_Amount', 'Received_Amount'],
                               title="Claimed vs Paid Amounts by Insurance Provider",
                               labels={'value': 'Amount', 'variable': 'Category'}, barmode='group')
        st.plotly_chart(bar_insurance)

        # Histogram: Monthly Claims
        hist_chart = px.histogram(filtered_df, x='Month', y='SUBMITTED_AMOUNT', color='Year',
                                  title="Monthly Claim Distribution", barmode='group',
                                  category_orders={'Month': month_order})
        st.plotly_chart(hist_chart)

        # Export to Excel (filtered data)
        output_file = "Insurance_Claims_Report.xlsx"
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            grouped_summary.to_excel(writer, sheet_name="Summary", index=False)
            grouped_submitted.to_excel(writer, sheet_name="Submitted Claims Per Month", index=False)
            doctor_insurance_month.to_excel(writer, sheet_name="Doctor_Insurance_Monthly", index=False)
            filtered_df.to_excel(writer, sheet_name="All Claims Raw (Filtered)", index=False)

        # Download Button
        with open(output_file, "rb") as file:
            st.download_button("Download Full Insurance Claims Excel Report (Filtered)",
                               file, file_name="Insurance_Claims_Report.xlsx")
        # Cleanup (optional, for local runs)
        os.remove(output_file)

    except Exception as e:
        st.error(f"Error processing file: {e}. Please check your data format and try again.")

Writing dashboard.py


In [2]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import os  # For file cleanup

st.set_page_config(page_title="Insurance Claims Dashboard", layout="wide")
st.title("Insurance Claims Analysis and Visualization Dashboard")

# File uploader with size warning
uploaded_file = st.file_uploader("Upload Excel/CSV File (Max 50MB recommended)", type=["csv", "xls", "xlsx", "xlsm"])

if uploaded_file:
    try:
        # Load data
        if uploaded_file.name.endswith(".csv"):
            df = pd.read_csv(uploaded_file)
        else:
            df = pd.read_excel(uploaded_file, engine='openpyxl')

        # Basic validation: Check for required columns
        required_cols = ['RA_RECEIVE_DATE', 'INSURANCE / TPA', 'PAID_AMOUNT']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            st.error(f"Missing required columns: {missing_cols}. Please check your file.")
            st.stop()

        # Data cleaning
        df.columns = df.columns.str.strip()

        # Convert amount columns to numeric (expanded for robustness)
        amount_cols = ['PAID_AMOUNT', 'ReSubmission_Paid_Amount_1', 'ReSubmission_Paid_Amount2',
                       'SUBMITTED_AMOUNT', 'ReSubmitted_Amount_1', 'ReSubmitted_Amount2',
                       'DENIED_AMOUNT', 'RESUBMISSION_DENIED_AMOUNT_RA_1', 'RESUBMISSION_DENIED_AMOUNT_RA_2']

        df[amount_cols] = df[amount_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

        # Convert 'RA_RECEIVE_DATE' column to datetime
        df['RA_RECEIVE_DATE'] = pd.to_datetime(df['RA_RECEIVE_DATE'], errors='coerce')
        df = df.dropna(subset=['RA_RECEIVE_DATE'])

        # Extract year, month, and quarter
        df['RA_RECEIVE_Year'] = df['RA_RECEIVE_DATE'].dt.year
        df['RA_RECEIVE_Month'] = df['RA_RECEIVE_DATE'].dt.strftime('%b')
        df['Quarter'] = df['RA_RECEIVE_DATE'].dt.quarter

        # Simplified calculations (adjust logic as needed for accuracy)
        df['Total Submitted Amount'] = round(df['SUBMITTED_AMOUNT'] + df['ReSubmitted_Amount_1'] + df['ReSubmitted_Amount2'], 2)
        df['Total Paid Amount'] = round(df['PAID_AMOUNT'] + df['ReSubmission_Paid_Amount_1'] + df['ReSubmission_Paid_Amount2'], 2)
        df['Total Denied Amount'] = round((df['DENIED_AMOUNT'] - df['ReSubmitted_Amount_1']) + (df['RESUBMISSION_DENIED_AMOUNT_RA_1'] - df['ReSubmitted_Amount2']) + df['RESUBMISSION_DENIED_AMOUNT_RA_2'], 2)
        df['Total Pending Amount'] = round(df['SUBMITTED_AMOUNT'] - (df['Total Paid Amount'] + df['Total Denied Amount']), 2)

        # Sidebar filters for interactivity
        st.sidebar.header("Filters")

        # Options for years with "ALL" added
        year_options = ["ALL"] + sorted(df['RA_RECEIVE_Year'].unique())
        selected_year = st.sidebar.multiselect("Select Year(s)", options=year_options, default=["ALL"])

        # Options for insurance with "ALL" added
        insurance_options = ["ALL"] + list(df['INSURANCE / TPA'].unique())
        selected_insurance = st.sidebar.multiselect("Select Insurance(s)", options=insurance_options, default=["ALL"])

        # Determine effective selections: if "ALL" is selected, use all options; otherwise, use selected ones
        effective_year = df['RA_RECEIVE_Year'].unique() if "ALL" in selected_year else [y for y in selected_year if y != "ALL"]
        effective_insurance = df['INSURANCE / TPA'].unique() if "ALL" in selected_insurance else [i for i in selected_insurance if i != "ALL"]

        # Apply filters using the effective selections
        filtered_df = df[
            (df['RA_RECEIVE_Year'].isin(effective_year)) &
            (df['INSURANCE / TPA'].isin(effective_insurance))
        ]

        # Group by year, insurance provider, and month (filtered)
        month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        grouped_paid = filtered_df.groupby(['RA_RECEIVE_Year', 'INSURANCE / TPA', 'RA_RECEIVE_Month'])['Total Paid Amount'].sum().unstack(fill_value=0)
        grouped_paid = grouped_paid.reindex(columns=month_order, fill_value=0).reset_index().sort_values(by='RA_RECEIVE_Year')

        # Additional summary table (new for completeness)
        summary_table = filtered_df.groupby(['RA_RECEIVE_Year', 'INSURANCE / TPA']).agg(
            Total_Submitted=('Total Submitted Amount', 'sum'),
            Total_Paid=('Total Paid Amount', 'sum'),
            Total_Denied=('Total Denied Amount', 'sum'),
            Total_Pending=('Total Pending Amount', 'sum')
        ).reset_index().sort_values(by='RA_RECEIVE_Year')

        # Display Tables
        st.subheader("Paid Amount Per Month (Filtered)")
        st.dataframe(grouped_paid)
        st.subheader("Summary Table: Submitted, Paid, Denied by Year and Insurance (Filtered)")
        st.dataframe(summary_table)

        # Charts Section
        st.subheader("Data Visualizations (Filtered Data)")
        if filtered_df.empty:
            st.warning("No data available for the selected filters. Please adjust your selections.")
        else:
            # Existing/Enhanced Charts
            st.markdown("### Trends and Comparisons")
            
            # Enhanced Bar Chart: Total Paid Amount per Year (added color by quarter for more insight)
            yearly_paid = filtered_df.groupby(['RA_RECEIVE_Year', 'Quarter'])['Total Paid Amount'].sum().reset_index()
            bar_fig = px.bar(yearly_paid, x='RA_RECEIVE_Year', y='Total Paid Amount', color='Quarter',
                             title="Yearly Paid Amount by Quarter", labels={'Total Paid Amount': "Total Paid ($)"},
                             color_discrete_sequence=px.colors.qualitative.Set1)
            st.plotly_chart(bar_fig)

            # Enhanced Bar Chart: Paid Amount by Insurance Provider (horizontal for readability)
            insurance_paid = filtered_df.groupby('INSURANCE / TPA')['Total Paid Amount'].sum().reset_index()
            bar_insurance = px.bar(insurance_paid, x='Total Paid Amount', y='INSURANCE / TPA', orientation='h',
                                   title="Paid Amount by Insurance Provider",
                                   labels={'Total Paid Amount': "Total Paid ($)"}, color_discrete_sequence=px.colors.qualitative.Set2)
            st.plotly_chart(bar_insurance)

            # Enhanced Histogram: Distribution of Paid Amounts (added marginal rug plot)
            hist_fig = px.histogram(filtered_df, x='Total Paid Amount', nbins=50,
                                    title="Distribution of Paid Amounts", labels={'Total Paid Amount': "Paid Amount ($)"},
                                    marginal="rug", color_discrete_sequence=['#1f77b4'])
            st.plotly_chart(hist_fig)

            # Enhanced Scatter Chart: Paid vs. Submitted by Insurance (added trendline)
            scatter_data = filtered_df.groupby('INSURANCE / TPA').agg(
                Total_Submitted=('Total Submitted Amount', 'sum'),
                Total_Paid=('Total Paid Amount', 'sum')
            ).reset_index()
            scatter_fig = px.scatter(scatter_data, x='Total_Submitted', y='Total_Paid', color='INSURANCE / TPA',
                                     size='Total_Paid', title="Insurance Performance: Submitted vs. Paid Amounts",
                                     labels={'Total_Submitted': 'Total Submitted ($)', 'Total_Paid': 'Total Paid ($)'},
                                     trendline="ols")
            st.plotly_chart(scatter_fig)


            st.markdown("### Time-Series and Trends")
            
            # Line Chart: Paid Amount Trends Over Time
            time_trend = filtered_df.groupby(['RA_RECEIVE_Year', 'RA_RECEIVE_Month'])['Total Paid Amount'].sum().reset_index()
            time_trend['Month-Year'] = time_trend['RA_RECEIVE_Month'] + '-' + time_trend['RA_RECEIVE_Year'].astype(str)
            line_fig = px.line(time_trend, x='Month-Year', y='Total Paid Amount',
                               title="Monthly Paid Amount Trends", labels={'Total Paid Amount': "Total Paid ($)"},
                               color_discrete_sequence=['#ff7f0e'])
            st.plotly_chart(line_fig)

            # Area Chart: Cumulative Paid Amounts by Insurance
            area_data = filtered_df.groupby('INSURANCE / TPA')['Total Paid Amount'].sum().reset_index().sort_values('Total Paid Amount', ascending=False)
            area_fig = px.area(area_data, x='INSURANCE / TPA', y='Total Paid Amount',
                               title="Cumulative Paid Amounts by Insurance Provider",
                               labels={'Total Paid Amount': "Total Paid ($)"}, color_discrete_sequence=px.colors.qualitative.Pastel)
            st.plotly_chart(area_fig)

            st.markdown("### Proportions and Distributions")
            
            # Pie Chart: Paid vs. Denied Proportions by Insurance
            pie_data = filtered_df.groupby('INSURANCE / TPA').agg(
                Total_Paid=('Total Paid Amount', 'sum'),
                Total_Denied=('Total Denied Amount', 'sum')
            ).reset_index()
            pie_data_melted = pie_data.melt(id_vars='INSURANCE / TPA', value_vars=['Total_Paid', 'Total_Denied'],
                                             var_name='Status', value_name='Amount')
            pie_fig = px.pie(pie_data_melted, values='Amount', names='Status', color='Status',
                             title="Paid vs. Denied Amounts by Insurance Provider",
                             labels={'Amount': 'Amount ($)'}, color_discrete_map={'Total_Paid': '00F7FF', 'Total_Denied': 'red'})
            st.plotly_chart(pie_fig)

            # Box Plot: Paid Amount Distribution by Insurance
            box_fig = px.box(filtered_df, x='INSURANCE / TPA', y='Total Paid Amount',
                             title="Paid Amount Distribution by Insurance Provider",
                             labels={'Total Paid Amount': "Paid Amount ($)"}, color='INSURANCE / TPA')
            st.plotly_chart(box_fig)

            st.markdown("### Advanced Insights")
            
            # Heatmap: Monthly Paid Amounts by Year and Insurance
            heatmap_data = filtered_df.groupby(['RA_RECEIVE_Year', 'RA_RECEIVE_Month', 'INSURANCE / TPA'])['Total Paid Amount'].sum().reset_index()
            heatmap_pivot = heatmap_data.pivot_table(values='Total Paid Amount', index=['RA_RECEIVE_Year', 'INSURANCE / TPA'], columns='RA_RECEIVE_Month', fill_value=0)
            heatmap_pivot = heatmap_pivot.reindex(columns=month_order, fill_value=0)
            heatmap_fig = px.imshow(heatmap_pivot, text_auto=True, aspect="auto",
                                    title="Heatmap of Monthly Paid Amounts by Year and Insurance",
                                    labels=dict(x="Month", y="Year & Insurance", color="Paid Amount ($)"))
            st.plotly_chart(heatmap_fig)

            # Scatter Plot: Paid vs. Denied by Insurance
            scatter_denied = filtered_df.groupby('INSURANCE / TPA').agg(
                Total_Paid=('Total Paid Amount', 'sum'),
                Total_Denied=('Total Denied Amount', 'sum')
            ).reset_index()
            scatter_denied_fig = px.scatter(scatter_denied, x='Total_Denied', y='Total_Paid', color='INSURANCE / TPA',
                                            size='Total_Paid', title="Paid vs. Denied Amounts by Insurance",
                                            labels={'Total_Denied': 'Total Denied ($)', 'Total_Paid': 'Total Paid ($)'})
            st.plotly_chart(scatter_denied_fig)

            # Stacked Bar Chart: Multi-Metric Breakdown by Year
            stacked_data = filtered_df.groupby('RA_RECEIVE_Year').agg(
                Total_Submitted=('Total Submitted Amount', 'sum'),
                Total_Paid=('Total Paid Amount', 'sum'),
                Total_Denied=('Total Denied Amount', 'sum'),
                Total_Pending=('Total Pending Amount', 'sum')
            ).reset_index()
            stacked_fig = px.bar(stacked_data, x='RA_RECEIVE_Year', y=['Total_Submitted', 'Total_Paid', 'Total_Denied', 'Total_Pending'],
                                 title="Submitted, Paid, Denied, and Pending Amounts by Year",
                                 labels={'value': 'Amount ($)', 'variable': 'Metric'}, barmode='stack')
            st.plotly_chart(stacked_fig)

            # Faceted Histogram: Paid Amounts by Quarter
            facet_hist_fig = px.histogram(filtered_df, x='Total Paid Amount', facet_col='Quarter', nbins=30,
                                          title="Distribution of Paid Amounts by Quarter",
                                          labels={'Total Paid Amount': "Paid Amount ($)"})
            st.plotly_chart(facet_hist_fig)

        # Export to Excel (filtered data, multiple sheets)
        output_file = "Insurance_Claims_Report.xlsx"
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            grouped_paid.to_excel(writer, sheet_name="Paid Claims Per Month", index=False)
            summary_table.to_excel(writer, sheet_name="Summary", index=False)
            filtered_df.to_excel(writer, sheet_name="All Claims Raw (Filtered)", index=False)

        # Download button for the Excel file
        with open(output_file, "rb") as file:
            st.download_button("Download Insurance Claims Report (Filtered)", file, file_name="Insurance_Claims_Report.xlsx")
        # Cleanup
        os.remove(output_file)

    except Exception as e:
        st.error(f"Error processing file: {e}. Please check your data format and try again.")

Overwriting dashboard.py


In [3]:
!ngrok config add-authtoken 2xs39RuD1UzlliD3ApqH63JGYwP_3cfctbzEY44H49mwHVgxG

Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import os

# Set your authtoken (Only run once in terminal, remove from script after that)
ngrok.set_auth_token("2xs39RuD1UzlliD3ApqH63JGYwP_3cfctbzEY44H49mwHVgxG")

# Run Streamlit in the background
os.system("streamlit run dashboard.py &")

# Expose Streamlit using Ngrok
rahuldey_url = ngrok.connect("http://localhost:8501")
print(f"Streamlit is running on {rahuldey_url.public_url}")






  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://10.0.0.127:8502
  External URL: http://4.240.39.192:8502

Streamlit is running on https://6ccb3e89e73e.ngrok-free.app
